<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Construct a matrix `A` from singular values and vectors.


In [ ]:
import numpy as np
from scipy import sparse, linalg, stats
from scipy.sparse.linalg import svds, aslinearoperator, LinearOperator

Construct a dense matrix `A` from singular values and vectors.


In [ ]:
rng = np.random.default_rng()
orthogonal = stats.ortho_group.rvs(10, random_state=rng)
s = [1e-3, 1, 2, 3, 4]  # non-zero singular values
u = orthogonal[:, :5]         # left singular vectors
vT = orthogonal[:, 5:].T      # right singular vectors
A = u @ np.diag(s) @ vT

With only four singular values/vectors, the SVD approximates the original
matrix.


In [ ]:
u4, s4, vT4 = svds(A, k=4)
A4 = u4 @ np.diag(s4) @ vT4
np.allclose(A4, A, atol=1e-3)

True

With all five non-zero singular values/vectors, we can reproduce
the original matrix more accurately.


In [ ]:
u5, s5, vT5 = svds(A, k=5)
A5 = u5 @ np.diag(s5) @ vT5
np.allclose(A5, A)

True

The singular values match the expected singular values.


In [ ]:
np.allclose(s5, s)

True

Since the singular values are not close to each other in this example,
every singular vector matches as expected up to a difference in sign.


In [ ]:
(np.allclose(np.abs(u5), np.abs(u)) and
 np.allclose(np.abs(vT5), np.abs(vT)))

True

The singular vectors are also orthogonal.


In [ ]:
(np.allclose(u5.T @ u5, np.eye(5)) and
 np.allclose(vT5 @ vT5.T, np.eye(5)))

True

If there are (nearly) multiple singular values, the corresponding
individual singular vectors may be unstable, but the whole invariant
subspace containing all such singular vectors is computed accurately
as can be measured by angles between subspaces via 'subspace_angles'.


In [ ]:
rng = np.random.default_rng()
s = [1, 1 + 1e-6]  # non-zero singular values
u, _ = np.linalg.qr(rng.standard_normal((99, 2)))
v, _ = np.linalg.qr(rng.standard_normal((99, 2)))
vT = v.T
A = u @ np.diag(s) @ vT
A = A.astype(np.float32)
u2, s2, vT2 = svds(A, k=2, random_state=rng)
np.allclose(s2, s)

True

The angles between the individual exact and computed singular vectors
may not be so small. To check use:


In [ ]:
(linalg.subspace_angles(u2[:, :1], u[:, :1]) +
 linalg.subspace_angles(u2[:, 1:], u[:, 1:]))

array([0.06562513])  # may vary

In [ ]:
(linalg.subspace_angles(vT2[:1, :].T, vT[:1, :].T) +
 linalg.subspace_angles(vT2[1:, :].T, vT[1:, :].T))

array([0.06562507])  # may vary

As opposed to the angles between the 2-dimensional invariant subspaces
that these vectors span, which are small for rights singular vectors


In [ ]:
linalg.subspace_angles(u2, u).sum() < 1e-6

True

as well as for left singular vectors.


In [ ]:
linalg.subspace_angles(vT2.T, vT.T).sum() < 1e-6

True

The next example follows that of 'sklearn.decomposition.TruncatedSVD'.


In [ ]:
rng = np.random.RandomState(0)
X_dense = rng.random(size=(100, 100))
X_dense[:, 2 * np.arange(50)] = 0
X = sparse.csr_matrix(X_dense)
_, singular_values, _ = svds(X, k=5, random_state=rng)
print(singular_values)

[ 4.3293...  4.4491...  4.5420...  4.5987... 35.2410...]

The function can be called without the transpose of the input matrix
ever explicitly constructed.


In [ ]:
rng = np.random.default_rng()
G = sparse.rand(8, 9, density=0.5, random_state=rng)
Glo = aslinearoperator(G)
_, singular_values_svds, _ = svds(Glo, k=5, random_state=rng)
_, singular_values_svd, _ = linalg.svd(G.toarray())
np.allclose(singular_values_svds, singular_values_svd[-4::-1])

True

The most memory efficient scenario is where neither
the original matrix, nor its transpose, is explicitly constructed.
Our example computes the smallest singular values and vectors
of 'LinearOperator' constructed from the numpy function 'np.diff' used
column-wise to be consistent with 'LinearOperator' operating on columns.


In [ ]:
diff0 = lambda a: np.diff(a, axis=0)

Let us create the matrix from 'diff0' to be used for validation only.


In [ ]:
n = 5  # The dimension of the space.
M_from_diff0 = diff0(np.eye(n))
print(M_from_diff0.astype(int))

[[-1  1  0  0  0]
 [ 0 -1  1  0  0]
 [ 0  0 -1  1  0]
 [ 0  0  0 -1  1]]

The matrix 'M_from_diff0' is bi-diagonal and could be alternatively
created directly by


In [ ]:
M = - np.eye(n - 1, n, dtype=int)
np.fill_diagonal(M[:,1:], 1)
np.allclose(M, M_from_diff0)

True

Its transpose


In [ ]:
print(M.T)

[[-1  0  0  0]
 [ 1 -1  0  0]
 [ 0  1 -1  0]
 [ 0  0  1 -1]
 [ 0  0  0  1]]

can be viewed as the incidence matrix; see
Incidence matrix, (2022, Nov. 19), Wikipedia, https://w.wiki/5YXU,
of a linear graph with 5 vertices and 4 edges. The 5x5 normal matrix
``M.T @ M`` thus is


In [ ]:
print(M.T @ M)

[[ 1 -1  0  0  0]
 [-1  2 -1  0  0]
 [ 0 -1  2 -1  0]
 [ 0  0 -1  2 -1]
 [ 0  0  0 -1  1]]

the graph Laplacian, while the actually used in 'svds' smaller size
4x4 normal matrix ``M @ M.T``


In [ ]:
print(M @ M.T)

[[ 2 -1  0  0]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [ 0  0 -1  2]]

is the so-called edge-based Laplacian; see
Symmetric Laplacian via the incidence matrix, in Laplacian matrix,
(2022, Nov. 19), Wikipedia, https://w.wiki/5YXW.

The 'LinearOperator' setup needs the options 'rmatvec' and 'rmatmat'
of multiplication by the matrix transpose ``M.T``, but we want to be
matrix-free to save memory, so knowing how ``M.T`` looks like, we
manually construct the following function to be
used in ``rmatmat=diff0t``.


In [ ]:
def diff0t(a):
    if a.ndim == 1:
        a = a[:,np.newaxis]  # Turn 1D into 2D array
    d = np.zeros((a.shape[0] + 1, a.shape[1]), dtype=a.dtype)
    d[0, :] = - a[0, :]
    d[1:-1, :] = a[0:-1, :] - a[1:, :]
    d[-1, :] = a[-1, :]
    return d

We check that our function 'diff0t' for the matrix transpose is valid.


In [ ]:
np.allclose(M.T, diff0t(np.eye(n-1)))

True

Now we setup our matrix-free 'LinearOperator' called 'diff0_func_aslo'
and for validation the matrix-based 'diff0_matrix_aslo'.


In [ ]:
def diff0_func_aslo_def(n):
    return LinearOperator(matvec=diff0,
                          matmat=diff0,
                          rmatvec=diff0t,
                          rmatmat=diff0t,
                          shape=(n - 1, n))
diff0_func_aslo = diff0_func_aslo_def(n)
diff0_matrix_aslo = aslinearoperator(M_from_diff0)

And validate both the matrix and its transpose in 'LinearOperator'.


In [ ]:
np.allclose(diff0_func_aslo(np.eye(n)),
            diff0_matrix_aslo(np.eye(n)))

True

In [ ]:
np.allclose(diff0_func_aslo.T(np.eye(n-1)),
            diff0_matrix_aslo.T(np.eye(n-1)))

True

Having the 'LinearOperator' setup validated, we run the solver.


In [ ]:
n = 100
diff0_func_aslo = diff0_func_aslo_def(n)
u, s, vT = svds(diff0_func_aslo, k=3, which='SM')

The singular values squared and the singular vectors are known
explicitly; see
Pure Dirichlet boundary conditions, in
Eigenvalues and eigenvectors of the second derivative,
(2022, Nov. 19), Wikipedia, https://w.wiki/5YX6,
since 'diff' corresponds to first
derivative, and its smaller size n-1 x n-1 normal matrix
``M @ M.T`` represent the discrete second derivative with the Dirichlet
boundary conditions. We use these analytic expressions for validation.


In [ ]:
se = 2. * np.sin(np.pi * np.arange(1, 4) / (2. * n))
ue = np.sqrt(2 / n) * np.sin(np.pi * np.outer(np.arange(1, n),
                             np.arange(1, 4)) / n)
np.allclose(s, se, atol=1e-3)

True

In [ ]:
print(np.allclose(np.abs(u), np.abs(ue), atol=1e-6))

True